In [19]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
train_gen = ImageDataGenerator(rescale=1/255.,horizontal_flip = True)
validation_gen = ImageDataGenerator(rescale = 1/255.,horizontal_flip = True)
test_gen = ImageDataGenerator(rescale = 1/255.,horizontal_flip = True)

In [3]:
train_images = train_gen.flow_from_directory(
                'eyes/training',
                target_size = (100,100),
                batch_size = 32)

validation_images = train_gen.flow_from_directory(
                'eyes/validation',
                target_size = (100,100),
                batch_size = 32)

test_images = test_gen.flow_from_directory(
                'eyes/testing',
                target_size = (100,100),
                batch_size = 32)

Found 1742 images belonging to 2 classes.
Found 486 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [4]:
print(train_images.class_indices)
print(test_images.class_indices)
print(validation_images.class_indices)

{'ClosedFace': 0, 'OpenFace': 1}
{'ClosedFace': 0, 'OpenFace': 1}
{'ClosedFace': 0, 'OpenFace': 1}


In [5]:
model = Sequential()
model.add(Conv2D(16,(3,3),input_shape=(100,100,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(.2))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(512,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1000,activation="relu"))
model.add(Dropout(.2))
model.add(Dense(128,activation="relu"))
model.add(Dropout(.2))
model.add(Dense(32,activation="relu"))
model.add(Dropout(.2))
model.add(Dense(2,activation="sigmoid"))

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 128)       36992     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 128)      

In [7]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [8]:
history = model.fit_generator(
                            train_images,
                            steps_per_epoch = 64,
                            epochs = 30,
                            validation_data = validation_images,
                            validation_steps = 15)

Epoch 1/30
64/64 [==============================] - 5s 71ms/step - loss: 0.6881 - accuracy: 0.5389 - val_loss: 0.6116 - val_accuracy: 0.6479
Epoch 2/30
64/64 [==============================] - 3s 44ms/step - loss: 0.6748 - accuracy: 0.5660 - val_loss: 0.6879 - val_accuracy: 0.5264
Epoch 3/30
64/64 [==============================] - 3s 42ms/step - loss: 0.6689 - accuracy: 0.5823 - val_loss: 0.4869 - val_accuracy: 0.6905
Epoch 4/30
64/64 [==============================] - 3s 43ms/step - loss: 0.6542 - accuracy: 0.6094 - val_loss: 0.6278 - val_accuracy: 0.6509
Epoch 5/30
64/64 [==============================] - 3s 43ms/step - loss: 0.6103 - accuracy: 0.6747 - val_loss: 0.5297 - val_accuracy: 0.7247
Epoch 6/30
64/64 [==============================] - 3s 43ms/step - loss: 0.4977 - accuracy: 0.7601 - val_loss: 0.3656 - val_accuracy: 0.7313
Epoch 7/30
64/64 [==============================] - 3s 45ms/step - loss: 0.4290 - accuracy: 0.8096 - val_loss: 0.2581 - val_accuracy: 0.8987
Epoch 8/30
64

In [9]:
scores = model.evaluate_generator(test_images)

In [10]:
print(scores[1])

0.9692307710647583


In [12]:
model.save("drowsy.h5")

In [24]:
model = tf.keras.models.load_model('drowsy.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("drowsy_model.tflite", "wb").write(tflite_model)

35835232